In [1]:
!pip install transformers datasets
!pip install nltk
!pip install keybert

!pip install faiss-gpu sentence_transformers

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import requests
from bs4 import BeautifulSoup

def download_gutenberg_book(book_id):
    url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-h/{book_id}-h.htm"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()
   
    text = ' '.join(text.split())
    return text


gutenberg_books = {
    11: download_gutenberg_book(11),  
    1342: download_gutenberg_book(1342),  
    1661: download_gutenberg_book(1661)  
}

In [6]:
from transformers import pipeline

In [8]:
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer,device='cuda')

def preprocess_book(text):
    text = re.sub(r'\s+', ' ', text).strip()
    
    chunks = []
    chunk_size = 1000
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i+chunk_size])
    
    return chunks

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize

def preprocess_book(text):
    
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.!?]', '', text)
    text = text.lower()
    
   
    sentences = sent_tokenize(text)
    
    
    chunks = []
    current_chunk = []
    token_count = 0
    
    for sentence in sentences:
        tokens = tokenizer.tokenize(sentence)
        if token_count + len(tokens) > 1000:
            chunk_text = ' '.join(current_chunk)
            chunks.append(chunk_text)
            current_chunk = []
            token_count = 0
        current_chunk.append(sentence)
        token_count += len(tokens)
    
    if current_chunk:
        chunk_text = ' '.join(current_chunk)
        chunks.append(chunk_text)
    
    return chunks

In [17]:
preprocessed_books = {book_id: preprocess_book(text) for book_id, text in gutenberg_books.items()}


In [18]:
corpus = [chunk for chunks in preprocessed_books.values() for chunk in chunks]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 56.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 171.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 102.5 MB/s eta 0:00:0000:0100:01


In [21]:

import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

In [22]:

model_name = 'all-MiniLM-L6-v2'  
embed_model = SentenceTransformer(model_name)


embeddings = embed_model.encode(corpus, show_progress_bar=True)

faiss.normalize_L2(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 18.6 MB/s eta 0:00:00


In [49]:
from keybert import KeyBERT

keyword_model = KeyBERT()



In [23]:
def retrieve_passages(query, top_k=3):
    query_vector = embed_model.encode([query])
    faiss.normalize_L2(query_vector)
    D, I = index.search(query_vector, top_k)
    return [corpus[i] for i in I[0]]

In [43]:
def generate_summary(text, max_length=150):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).to('cuda')
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [44]:
def rag_summarize_concat(text):
    initial_summary = generate_summary(text)
    retrieved_passages = retrieve_passages(initial_summary)
    combined_text = initial_summary + " " + " ".join(retrieved_passages)
    final_summary = generate_summary(combined_text, max_length=200)
    return final_summary

In [52]:
def rag_summarize_extracted(text):
    initial_summary = generate_summary(text)
    retrieved_passages = retrieve_passages(initial_summary)
    
    keywords = []
    for passage in retrieved_passages:
        keywords.extend([kw for kw, _ in keyword_model.extract_keywords(passage, top_n=5, stop_words='english')])
    
    unique_keywords = list(set(keywords))
    augmented_text = initial_summary + " Additional key information: " + ", ".join(unique_keywords)
    
    final_summary = generate_summary(augmented_text, max_length=200)
    return final_summary


def rag_summarize_guided(text):
    initial_summary = generate_summary(text)
    retrieved_passages = retrieve_passages(initial_summary)
    
    prompt = f"Summarize the following text, focusing on these key points: {initial_summary}\n\nAdditional context:\n"
    for i, passage in enumerate(retrieved_passages, 1):
        prompt += f"{i}. {passage}\n"
    
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to('cuda')
    guided_summary_ids = model.generate(inputs["input_ids"], max_length=200, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(guided_summary_ids[0], skip_special_tokens=True)


In [28]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 2.1 MB/s eta 0:00:00 0:00:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=ac1062fd4ea16dd7862c3d3fb473d8813912322e1c3d5a57af8e79b97e93201d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [29]:
from datasets import load_dataset
from rouge_score import rouge_scorer
import numpy as np

booksum_dataset = load_dataset("kmfoda/booksum", split="test")
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1484 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1431 [00:00<?, ? examples/s]

In [53]:


results = {'baseline': [], 'rag_concat': [], 'rag_extracted': [], 'rag_guided': []}

for example in booksum_dataset:
    
    reference_summary = example['summary']
    
    
    baseline_summary = generate_summary(example['chapter'])
    results['baseline'].append(scorer.score(reference_summary, baseline_summary))
    
    # RAG - Concatenation
    rag_concat_summary = rag_summarize_concat(example['chapter'])
    results['rag_concat'].append(scorer.score(reference_summary, rag_concat_summary))
    
    # RAG - Extracted Information
    rag_extracted_summary = rag_summarize_extracted(example['chapter'])
    results['rag_extracted'].append(scorer.score(reference_summary, rag_extracted_summary))
    
    # RAG - Guided Generation
    rag_guided_summary = rag_summarize_guided(example['chapter'])
    results['rag_guided'].append(scorer.score(reference_summary, rag_guided_summary))

# Print results
for method in results:
    print(f"{method.capitalize()} Average ROUGE Scores:")
    for metric in ['rouge1', 'rouge2', 'rougeL']:
        avg_score = np.mean([score[metric].fmeasure for score in results[method]])
        print(f"{metric}: {avg_score:.4f}")
    print()

Baseline Average ROUGE Scores:
rouge1: 0.1107
rouge2: 0.0178
rougeL: 0.0716

Rag_concat Average ROUGE Scores:
rouge1: 0.1087
rouge2: 0.0155
rougeL: 0.0704

Rag_extracted Average ROUGE Scores:
rouge1: 0.1010
rouge2: 0.0162
rougeL: 0.0670

Rag_guided Average ROUGE Scores:
rouge1: 0.1181
rouge2: 0.0167
rougeL: 0.0752



In [58]:
booksum_dataset[0].keys()

dict_keys(['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length'])

In [59]:
from datasets import load_dataset
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import torch
from rouge_score import rouge_scorer
import numpy as np

# Load the dataset
full_dataset = load_dataset("kmfoda/booksum", split="train")

# Split the dataset
train_test_split = full_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [60]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

Map:   0%|          | 0/8640 [00:00<?, ? examples/s]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

In [61]:
def preprocess_function(examples):
    inputs = tokenizer(examples["chapter"], max_length=1024, truncation=True, padding="max_length")
    targets = tokenizer(examples["summary"], max_length=256, truncation=True, padding="max_length")
    
    inputs["labels"] = targets["input_ids"]
    return inputs

In [64]:
!pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 3.5 MB/s eta 0:00:00a 0:00:01


In [65]:


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(pred, label) for pred, label in zip(decoded_preds, decoded_labels)]
    
    
    rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
    rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])
    rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])
    
    return {
        'rouge1': rouge1,
        'rouge2': rouge2,
        'rougeL': rougeL,
    }


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
)


early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)



trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`